In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
def simulate_season(dados_treino):
    df = dados_treino
    data1 = (
        df[
            [
                "HomeTeam",
                "AwayTeam",
                "HomeTeam_elo",
                "AwayTeam_elo",
                "Divisao",
                "HomeGoals",
            ]
        ]
        .assign(home=1)
        .rename(
            columns={
                "HomeTeam": "team",
                "AwayTeam": "opponent",
                "HomeTeam_elo": "team_elo",
                "AwayTeam_elo": "opponent_elo",
                "Divisao": "Divisao",
                "HomeGoals": "goals",
            }
        )
    )
    data2 = (
        df[
            [
                "AwayTeam",
                "HomeTeam",
                "AwayTeam_elo",
                "HomeTeam_elo",
                "Divisao",
                "AwayGoals",
            ]
        ]
        .assign(home=0)
        .rename(
            columns={
                "AwayTeam": "team",
                "HomeTeam": "opponent",
                "AwayTeam_elo": "team_elo",
                "HomeTeam_elo": "opponent_elo",
                "Divisao": "Divisao",
                "AwayGoals": "goals",
            }
        )
    )
    data = pd.concat([data1, data2])
    poisson_model = smf.glm(
        formula="goals ~ home + team + opponent ",
        data=data,
        family=sm.families.Poisson(),
    ).fit()
    times = np.unique(data["team"])
    colunas = [
        "Time",
        "Pontos",
        "Jogos",
        "Vitórias",
        "Empates",
        "Derrotas",
        "Gols Pró",
        "Gols Contra",
        "Saldo de Gols",
    ]
    tabela_simulacao = pd.DataFrame(index=times, columns=colunas)
    tabela_simulacao = tabela_simulacao.fillna(0)
    tabela_simulacao["Time"] = times
    for i in times:
        for j in times:
            if i != j:
                home_team = i
                away_team = j
                home_team_elo = df[df["HomeTeam"] == home_team]
                home_team_elo = home_team_elo["HomeTeam_elo"].mean()
                away_team_elo = df[df["AwayTeam"] == away_team]
                away_team_elo = away_team_elo["AwayTeam_elo"].mean()
                home_score_rate = poisson_model.predict(
                    pd.DataFrame(
                        data={
                            "team": home_team,
                            "opponent": away_team,
                            "team_elo": home_team_elo,
                            "opponent_elo": away_team_elo,
                            "home": 1,
                        },
                        index=[1],
                    )
                )
                away_score_rate = poisson_model.predict(
                    pd.DataFrame(
                        data={
                            "team": away_team,
                            "opponent": home_team,
                            "team_elo": away_team_elo,
                            "opponent_elo": home_team_elo,
                            "home": 0,
                        },
                        index=[1],
                    )
                )
                home_goals = np.random.poisson(home_score_rate)
                away_goals = np.random.poisson(away_score_rate)
                tabela_simulacao.loc[i, "Jogos"] = tabela_simulacao.loc[i, "Jogos"] + 1
                tabela_simulacao.loc[j, "Jogos"] = tabela_simulacao.loc[j, "Jogos"] + 1
                tabela_simulacao.loc[i, "Gols Pró"] = (
                    tabela_simulacao.loc[i, "Gols Pró"] + home_goals
                )
                tabela_simulacao.loc[i, "Gols Contra"] = (
                    tabela_simulacao.loc[i, "Gols Contra"] + away_goals
                )
                tabela_simulacao.loc[j, "Gols Pró"] = (
                    tabela_simulacao.loc[j, "Gols Pró"] + away_goals
                )
                tabela_simulacao.loc[j, "Gols Contra"] = (
                    tabela_simulacao.loc[j, "Gols Contra"] + home_goals
                )
                tabela_simulacao.loc[i, "Saldo de Gols"] = (
                    tabela_simulacao.loc[i, "Saldo de Gols"] + home_goals - away_goals
                )
                tabela_simulacao.loc[j, "Saldo de Gols"] = (
                    tabela_simulacao.loc[j, "Saldo de Gols"] - home_goals + away_goals
                )

                if home_goals > away_goals:
                    tabela_simulacao.loc[i, "Vitórias"] = (
                        tabela_simulacao.loc[i, "Vitórias"] + 1
                    )
                    tabela_simulacao.loc[j, "Derrotas"] = (
                        tabela_simulacao.loc[j, "Derrotas"] + 1
                    )
                    tabela_simulacao.loc[i, "Pontos"] = (
                        tabela_simulacao.loc[i, "Pontos"] + 3
                    )
                elif home_goals < away_goals:
                    tabela_simulacao.loc[j, "Vitórias"] = (
                        tabela_simulacao.loc[j, "Vitórias"] + 1
                    )
                    tabela_simulacao.loc[i, "Derrotas"] = (
                        tabela_simulacao.loc[i, "Derrotas"] + 1
                    )
                    tabela_simulacao.loc[j, "Pontos"] = (
                        tabela_simulacao.loc[j, "Pontos"] + 3
                    )
                else:
                    tabela_simulacao.loc[j, "Empates"] = (
                        tabela_simulacao.loc[j, "Empates"] + 1
                    )
                    tabela_simulacao.loc[i, "Empates"] = (
                        tabela_simulacao.loc[i, "Empates"] + 1
                    )
                    tabela_simulacao.loc[i, "Pontos"] = (
                        tabela_simulacao.loc[i, "Pontos"] + 1
                    )
                    tabela_simulacao.loc[j, "Pontos"] = (
                        tabela_simulacao.loc[j, "Pontos"] + 1
                    )

    tabela_simulacao = tabela_simulacao.sort_values(
        by=["Pontos", "Vitórias", "Saldo de Gols", "Gols Pró"],
        axis="index",
        ascending=False,
    )
    tabela_simulacao
    return tabela_simulacao

In [3]:
dados_treino = pd.read_csv("elo_compnat/data/brasileirao.csv")
simulate_season(dados_treino)

KeyError: "['HomeTeam_elo', 'AwayTeam_elo'] not in index"